# Compute sessions last ts

In .env

```
EXTRACTOR_SECRET_KEY=
EXTRACTOR_URL=
MONGODB_URL=
```

Later on, in ORGS_TO_PROCESS, put an org id in it. 

In [1]:
# Set current working dir
import os
from dotenv import load_dotenv

# Path with .env
os.chdir("/Users/nicolasoulianov/phospho/phospho/backend/scripts")

load_dotenv()

True

In [2]:
import pymongo

# Initialize pymongo client
client = pymongo.MongoClient(os.getenv('MONGODB_URL'))
db = client[os.getenv('MONGODB_NAME')]

In [3]:
db.name

'production'

In [4]:
db["sessions"].create_index(
                [("last_message_ts", pymongo.DESCENDING)]
            )

'last_message_ts_-1'

In [5]:
results = db["tasks"].aggregate(
    [
        {
            "$match": {
                "$and": [
                    {"session_id": {"$exists": True}},
                    {"session_id": {"$ne": None}},
                ]
            }
        },
        # Group by session_id and get the max created_at of the tasks
        {"$group": {"_id": "$session_id", "last_message_ts": {"$max": "$created_at"}}},
        # Project
        {
            "$project": {
                "_id": 0,
                "id": "$_id",
                "last_message_ts": 1,
            }
        },
        # Merge into the sessions (update the last_message_ts)
        {
            "$merge": {
                "into": "sessions",
                "on": "id",
                # When matched, update the filed
                "whenMatched": "merge",
                "whenNotMatched": "discard",
            }
        },
    ]
)